In [4]:
import re
import sys
import pandas as pd
import numpy as np
from github import Github
from getpass import getpass
from pandas.io.json import json_normalize
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import networkx as nx
import pickle
%matplotlib inline

In [5]:
repos = pd.read_csv("awe_repos_100_in_cats.csv", encoding="utf-8")

In [6]:
ACCESS_TOKEN = getpass(prompt="Enter github token:")
client = Github(ACCESS_TOKEN, per_page=100)

Enter github token:········


In [7]:
#g = nx.DiGraph() #or read it from disk 
g = nx.read_gpickle("github.BAK.gpickle")
cache = {
    'repos':{},
    'contributors':{},
    'followers':{}
}

In [8]:
mtsw_repo = [n for n in g if g.node[n]['type'] == 'repo']
len(mtsw_repo)

12291

In [ ]:
%%time

START_FROM = 12291+1 #index
for i, repo_name in enumerate(repos.iloc[START_FROM:]["repo"].values):
    if ((START_FROM+i) % 100) == 0: 
        print(START_FROM+i,repo_name)
        nx.write_gpickle(g, "github.BAK.gpickle")
    try:
        repo = client.get_repo(repo_name)
        g.add_node(repo.name + '(repo)', 
                   type='repo', 
                   lang=repo.language, 
                   owner=repo.owner.login,
                   description=repo.description
                  )
        cache['repos'][repo_name] = repo
        
        for contributor in repo.get_contributors():
            g.add_node(contributor.login + '(user)', type='user')
            g.add_edge(contributor.login + '(user)', repo.name + '(repo)', type='contributors')
            cache['contributors'][contributor.login] = contributor
            for follower in contributor.get_followers():
                if follower.login + '(user)' in g:
                    g.add_edge(follower.login + '(user)', contributor.login + '(user)', 
                           type='follows')
                    cache['followers'][follower.login] = follower
        
    except Exception, e: #ssl.SSLError
        print >> sys.stderr, "Encountered an error fetching data for", \
                             repo_name, "Skipping."
        print >> sys.stderr, e

    #if ((START_FROM+i) % 100) == 0: print "Processed", START_FROM+i+1, " ", repo_name," nodes. Num nodes/edges in graph", \
    #      g.number_of_nodes(), "/", g.number_of_edges()
    if client.rate_limiting < 400: print("Rate limit remaining", client.rate_limiting)
    
nx.write_gpickle(g, "github.BAK.gpickle")    
pickle.dump( cache, open( "cache.github.BAK.pickle", "wb" ) )

In [97]:
len(cache["repos"])

8087

In [ ]:
pickle.dump( cache, open( "cache.github.pickle", "wb" ) )

In [100]:
nx.info(g)

'Name: \nType: DiGraph\nNumber of nodes: 95939\nNumber of edges: 721184\nAverage in degree:   7.5171\nAverage out degree:   7.5171'

In [99]:
nx.write_gpickle(g, "github.LONG2.gpickle")

In [62]:
mtsw_users = [n for n in g if g.node[n]['type'] == 'user']
h = g.subgraph(mtsw_users)

#nx.write_gexf(h, "github2.gexf")
#nx.write_gml(h, "github2.gml")
nx.write_graphml(h, "github1770.graphml")

In [101]:
res = []
for name, _repo in cache["repos"].iteritems():
    df = json_normalize(_repo.raw_data)
    res.append(df)
repos_df = pd.concat(res, ignore_index=True)

In [102]:
cache["repos"].iteritems().next()

(u'MADEAPPS/newton-dynamics', <github.Repository.Repository at 0x7f35a745b250>)

In [103]:
repos_df.to_csv("repos8087.csv", encoding="utf-8")

Encountered an error fetching data for substack/stream-combiner2 Skipping.
('_ssl.c:565: The handshake operation timed out',)
Encountered an error fetching data for nodejs/readable-stream Skipping.
timed out
Encountered an error fetching data for almost/through2-concurrent Skipping.
timed out
Encountered an error fetching data for sockjs/sockjs-node Skipping.
timed out
Encountered an error fetching data for SocketCluster/socketcluster Skipping.
timed out
Encountered an error fetching data for primus/primus Skipping.
timed out
Encountered an error fetching data for simonswain/straw Skipping.
timed out
Encountered an error fetching data for fed135/Kalm Skipping.
timed out
Encountered an error fetching data for lovell/sharp Skipping.
timed out
Encountered an error fetching data for sindresorhus/image-type Skipping.
timed out
Encountered an error fetching data for aheckmann/gm Skipping.
timed out
Encountered an error fetching data for EyalAr/lwip Skipping.
timed out
Encountered an error fetching data for nodeca/pica Skipping.
timed out
Encountered an error fetching data for sindresorhus/is-progressive Skipping.
timed out
Encountered an error fetching data for nodeca/probe-image-size Skipping.
timed out
Encountered an error fetching data for epeli/underscore.string Skipping.
timed out
Encountered an error fetching data for ashtuchkin/iconv-lite Skipping.
timed out
Encountered an error fetching data for sindresorhus/repeating Skipping.
timed out
Encountered an error fetching data for sindresorhus/string-length Skipping.
timed out
Encountered an error fetching data for sindresorhus/camelcase Skipping.
timed out
Encountered an error fetching data for sindresorhus/escape-string-regexp Skipping.
timed out
Encountered an error fetching data for sindresorhus/execall Skipping.
timed out
Encountered an error fetching data for sindresorhus/splice-string Skipping.
timed out
Encountered an error fetching data for sindresorhus/indent-string Skipping.
timed out
Encountered an error fetching data for sindresorhus/strip-indent Skipping.
timed out
Encountered an error fetching data for sindresorhus/detect-indent Skipping.
timed out
Encountered an error fetching data for mathiasbynens/he Skipping.
timed out
Encountered an error fetching data for mashpie/i18n-node Skipping.
timed out
Encountered an error fetching data for nodeca/babelfish Skipping.
timed out
Encountered an error fetching data for sindresorhus/parse-columns Skipping.
timed out
Encountered an error fetching data for codekirei/hanging-indent Skipping.
timed out
Encountered an error fetching data for sindresorhus/matcher Skipping.
timed out
Encountered an error fetching data for sindresorhus/random-int Skipping.
timed out
Encountered an error fetching data for sindresorhus/random-float Skipping.
timed out
Encountered an error fetching data for sindresorhus/unique-random Skipping.
timed out
Encountered an error fetching data for sindresorhus/round-to Skipping.
timed out
Encountered an error fetching data for scijs/ndarray Skipping.
timed out
Encountered an error fetching data for josdejong/mathjs Skipping.
timed out
Encountered an error fetching data for sindresorhus/math-sum Skipping.
timed out
Encountered an error fetching data for sindresorhus/math-clamp Skipping.
timed out
Encountered an error fetching data for fibo/algebra Skipping.
timed out
Encountered an error fetching data for felixge/node-dateformat Skipping.
timed out
Encountered an error fetching data for samverschueren/tz-format Skipping.
timed out
Encountered an error fetching data for sindresorhus/normalize-url Skipping.
timed out
Encountered an error fetching data for sindresorhus/humanize-url Skipping.
timed out
Encountered an error fetching data for nodeca/url-unshort Skipping.
timed out
Encountered an error fetching data for pid/speakingurl Skipping.
timed out
Encountered an error fetching data for snd/url-pattern Skipping.
timed out
In [58]:
Encountered an error fetching data for wooorm/remark Skipping.
('The read operation timed out',)
Encountered an error fetching data for inikulin/parse5 Skipping.
timed out
Encountered an error fetching data for sindresorhus/strip-json-comments Skipping.
timed out
Encountered an error fetching data for sindresorhus/strip-css-comments Skipping.
timed out
Encountered an error fetching data for sindresorhus/parse-json Skipping.
timed out
Encountered an error fetching data for medialize/URI.js Skipping.
timed out
Encountered an error fetching data for postcss/postcss Skipping.
timed out
Encountered an error fetching data for dominictarr/JSONStream Skipping.
timed out
Encountered an error fetching data for mafintosh/csv-parser Skipping.
timed out
Encountered an error fetching data for sindresorhus/neat-csv Skipping.
timed out
Encountered an error fetching data for pegjs/pegjs Skipping.
timed out
Encountered an error fetching data for lapwinglabs/x-ray Skipping.
timed out
Encountered an error fetching data for Hardmath123/nearley Skipping.
timed out
Encountered an error fetching data for juliangruber/binary-extract Skipping.
timed out
Encountered an error fetching data for nemtsov/json-mask Skipping.
timed out
Encountered an error fetching data for stylecow/stylecow Skipping.
timed out
Encountered an error fetching data for nodeca/js-yaml Skipping.
timed out
Encountered an error fetching data for dominictarr/excel-stream Skipping.
timed out
Encountered an error fetching data for seegno/google-libphonenumber Skipping.
timed out
Encountered an error fetching data for sindresorhus/pretty-bytes Skipping.
timed out
Encountered an error fetching data for sindresorhus/pretty-ms Skipping.
timed out
Encountered an error fetching data for rauchg/ms.js Skipping.
timed out
Encountered an error fetching data for AriaMinaei/pretty-error Skipping.
timed out
Encountered an error fetching data for taijinlee/humanize Skipping.
timed out
Encountered an error fetching data for archiverjs/node-archiver Skipping.
('_ssl.c:565: The handshake operation timed out',)
Encountered an error fetching data for bower/decompress-zip Skipping.
timed out
Encountered an error fetching data for nodeca/pako Skipping.
timed out
Encountered an error fetching data for mafintosh/tar-stream Skipping.
timed out
Encountered an error fetching data for mafintosh/tar-fs Skipping.
timed out
Encountered an error fetching data for kevva/decompress Skipping.
timed out


Encountered an error fetching data for nodeca/pako Skipping.
timed out
Encountered an error fetching data for mafintosh/tar-stream Skipping.
timed out
Encountered an error fetching data for mafintosh/tar-fs Skipping.
timed out
Encountered an error fetching data for kevva/decompress Skipping.
timed out
Encountered an error fetching data for Level/levelup Skipping.
('_ssl.c:565: The handshake operation timed out',)
Encountered an error fetching data for mongodb/node-mongodb-native Skipping.
timed out
Encountered an error fetching data for felixge/node-mysql Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
(300, u'aerospike/aerospike-client-nodejs')
Encountered an error fetching data for balderdashy/waterline Skipping.
('The read operation timed out',)
Encountered an error fetching data for SierraSoftworks/Iridium Skipping.
timed out
Encountered an error fetching data for PhilWaldmann/openrecord Skipping.
timed out
Encountered an error fetching data for isaacs/node-tap Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
Encountered an error fetching data for substack/tape Skipping.
('_ssl.c:565: The handshake operation timed out',)
Encountered an error fetching data for mantoni/mochify.js Skipping.
timed out
Encountered an error fetching data for gotwarlost/istanbul Skipping.
('The read operation timed out',)
Encountered an error fetching data for bcoe/nyc Skipping.
timed out
Encountered an error fetching data for sinonjs/sinon Skipping.
('The read operation timed out',)
Encountered an error fetching data for nodeca/navit Skipping.
timed out
Encountered an error fetching data for theintern/intern Skipping.
timed out

Encountered an error fetching data for tj/node-progress Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
Encountered an error fetching data for bucaran/fly Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
Encountered an error fetching data for voodootikigod/node-serialport Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
(200, u'mafintosh/peek-stream')
Encountered an error fetching data for substack/stream-combiner2 Skipping.
('_ssl.c:565: The handshake operation timed out',)
Encountered an error fetching data for nodejs/readable-stream Skipping.
timed out
Encountered an error fetching data for almost/through2-concurrent Skipping.
timed out
Encountered an error fetching data for sockjs/sockjs-node Skipping.
timed out
Encountered an error fetching data for SocketCluster/socketcluster Skipping.
timed out
Encountered an error fetching data for primus/primus Skipping.
timed out
Encountered an error fetching data for simonswain/straw Skipping.
timed out
Encountered an error fetching data for fed135/Kalm Skipping.
timed out
Encountered an error fetching data for lovell/sharp Skipping.
timed out
Encountered an error fetching data for sindresorhus/image-type Skipping.
timed out
Encountered an error fetching data for aheckmann/gm Skipping.
timed out
Encountered an error fetching data for EyalAr/lwip Skipping.
timed out
Encountered an error fetching data for nodeca/pica Skipping.
timed out
Encountered an error fetching data for sindresorhus/is-progressive Skipping.
timed out
Encountered an error fetching data for nodeca/probe-image-size Skipping.
timed out
Encountered an error fetching data for epeli/underscore.string Skipping.
timed out
Encountered an error fetching data for ashtuchkin/iconv-lite Skipping.
timed out
Encountered an error fetching data for sindresorhus/repeating Skipping.
timed out
Encountered an error fetching data for sindresorhus/string-length Skipping.
timed out
Encountered an error fetching data for sindresorhus/camelcase Skipping.
timed out
Encountered an error fetching data for sindresorhus/escape-string-regexp Skipping.
timed out
Encountered an error fetching data for sindresorhus/execall Skipping.
timed out
Encountered an error fetching data for sindresorhus/splice-string Skipping.
timed out
Encountered an error fetching data for sindresorhus/indent-string Skipping.
timed out
Encountered an error fetching data for sindresorhus/strip-indent Skipping.
timed out
Encountered an error fetching data for sindresorhus/detect-indent Skipping.
timed out
Encountered an error fetching data for mathiasbynens/he Skipping.
timed out
Encountered an error fetching data for mashpie/i18n-node Skipping.
timed out
Encountered an error fetching data for nodeca/babelfish Skipping.
timed out
Encountered an error fetching data for sindresorhus/parse-columns Skipping.
timed out
Encountered an error fetching data for codekirei/hanging-indent Skipping.
timed out
Encountered an error fetching data for sindresorhus/matcher Skipping.
timed out
Encountered an error fetching data for sindresorhus/random-int Skipping.
timed out
Encountered an error fetching data for sindresorhus/random-float Skipping.
timed out
Encountered an error fetching data for sindresorhus/unique-random Skipping.
timed out
Encountered an error fetching data for sindresorhus/round-to Skipping.
timed out
Encountered an error fetching data for scijs/ndarray Skipping.
timed out
Encountered an error fetching data for josdejong/mathjs Skipping.
timed out
Encountered an error fetching data for sindresorhus/math-sum Skipping.
timed out
Encountered an error fetching data for sindresorhus/math-clamp Skipping.
timed out
Encountered an error fetching data for fibo/algebra Skipping.
timed out
Encountered an error fetching data for felixge/node-dateformat Skipping.
timed out
Encountered an error fetching data for samverschueren/tz-format Skipping.
timed out
Encountered an error fetching data for sindresorhus/normalize-url Skipping.
timed out
Encountered an error fetching data for sindresorhus/humanize-url Skipping.
timed out
Encountered an error fetching data for nodeca/url-unshort Skipping.
timed out
Encountered an error fetching data for pid/speakingurl Skipping.
timed out
Encountered an error fetching data for snd/url-pattern Skipping.
timed out
Encountered an error fetching data for wooorm/remark Skipping.
('The read operation timed out',)
Encountered an error fetching data for inikulin/parse5 Skipping.
timed out
Encountered an error fetching data for sindresorhus/strip-json-comments Skipping.
timed out
Encountered an error fetching data for sindresorhus/strip-css-comments Skipping.
timed out
Encountered an error fetching data for sindresorhus/parse-json Skipping.
timed out
Encountered an error fetching data for medialize/URI.js Skipping.
timed out
Encountered an error fetching data for postcss/postcss Skipping.
timed out
Encountered an error fetching data for dominictarr/JSONStream Skipping.
timed out
Encountered an error fetching data for mafintosh/csv-parser Skipping.
timed out
Encountered an error fetching data for sindresorhus/neat-csv Skipping.
timed out
Encountered an error fetching data for pegjs/pegjs Skipping.
timed out
Encountered an error fetching data for lapwinglabs/x-ray Skipping.
timed out
Encountered an error fetching data for Hardmath123/nearley Skipping.
timed out
Encountered an error fetching data for juliangruber/binary-extract Skipping.
timed out
Encountered an error fetching data for nemtsov/json-mask Skipping.
timed out
Encountered an error fetching data for stylecow/stylecow Skipping.
timed out
Encountered an error fetching data for nodeca/js-yaml Skipping.
timed out
Encountered an error fetching data for dominictarr/excel-stream Skipping.
timed out
Encountered an error fetching data for seegno/google-libphonenumber Skipping.
timed out
Encountered an error fetching data for sindresorhus/pretty-bytes Skipping.
timed out
Encountered an error fetching data for sindresorhus/pretty-ms Skipping.
timed out
Encountered an error fetching data for rauchg/ms.js Skipping.
timed out
Encountered an error fetching data for AriaMinaei/pretty-error Skipping.
timed out
Encountered an error fetching data for taijinlee/humanize Skipping.
timed out
Encountered an error fetching data for archiverjs/node-archiver Skipping.
('_ssl.c:565: The handshake operation timed out',)
Encountered an error fetching data for bower/decompress-zip Skipping.
timed out
Encountered an error fetching data for nodeca/pako Skipping.
timed out
Encountered an error fetching data for mafintosh/tar-stream Skipping.
timed out
Encountered an error fetching data for mafintosh/tar-fs Skipping.
timed out
Encountered an error fetching data for kevva/decompress Skipping.
timed out
Encountered an error fetching data for Level/levelup Skipping.
('_ssl.c:565: The handshake operation timed out',)
Encountered an error fetching data for mongodb/node-mongodb-native Skipping.
timed out
Encountered an error fetching data for felixge/node-mysql Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
(300, u'aerospike/aerospike-client-nodejs')
Encountered an error fetching data for balderdashy/waterline Skipping.
('The read operation timed out',)
Encountered an error fetching data for SierraSoftworks/Iridium Skipping.
timed out
Encountered an error fetching data for PhilWaldmann/openrecord Skipping.
timed out
Encountered an error fetching data for isaacs/node-tap Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
Encountered an error fetching data for substack/tape Skipping.
('_ssl.c:565: The handshake operation timed out',)
Encountered an error fetching data for mantoni/mochify.js Skipping.
timed out
Encountered an error fetching data for gotwarlost/istanbul Skipping.
('The read operation timed out',)
Encountered an error fetching data for bcoe/nyc Skipping.
timed out
Encountered an error fetching data for sinonjs/sinon Skipping.
('The read operation timed out',)
Encountered an error fetching data for nodeca/navit Skipping.
timed out
Encountered an error fetching data for theintern/intern Skipping.
timed out
Encountered an error fetching data for andris9/Nodemailer Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
Encountered an error fetching data for atom/electron Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
(400, u'yeoman/stringify-object')
Encountered an error fetching data for maxogden/art-of-node/#the-art-of-node Skipping.
404 {u'documentation_url': u'https://developer.github.com/v3', u'message': u'Not Found'}
Encountered an error fetching data for vsouza/awesome-ios/blob/master/.github/CONTRIBUTING.md Skipping.
404 {u'documentation_url': u'https://developer.github.com/v3', u'message': u'Not Found'}
(500, u'abbiio/iosdk')
Encountered an error fetching data for yapstudios/YapDatabase Skipping.
('The read operation timed out',)
(600, u'ArtSabintsev/Zephyr')
Encountered an error fetching data for pepibumur/SugarRecord Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
(700, u'ytakzk/Fusuma')
Encountered an error fetching data for square/Aardvark/ Skipping.
404 {u'documentation_url': u'https://developer.github.com/v3', u'message': u'Not Found'}
(800, u'evgenyneu/sound-fader-ios')
Encountered an error fetching data for comyarzaheri/TuningFork Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
Encountered an error fetching data for wangjwchn/JWAnimatedImage Skipping.
unsupported operand type(s) for +: 'NoneType' and 'str'
Encountered an error fetching data for vinbhai4u/CLImageViewPopup/ Skipping.
404 {u'documentation_url': u'https://developer.github.com/v3', u'message': u'Not Found'}
(900, u'eBay/NMessenger')
Encountered an error fetching data for RestKit/RestKit Skipping.
('_ssl.c:565: The handshake operation timed out',)
Encountered an error fetching data for foursquare/FSNetworking Skipping.
[Errno -2] Name or service not known
Encountered an error fetching data for fastlane/fastlane/tree/master/pem Skipping.
404 {u'documentation_url': u'https://developer.github.com/v3', u'message': u'Not Found'}
**OUTPUT MUTED**